In [1]:
import os
import numpy as np
import pandas as pd

In [65]:
sample_id = 'NA12878'
chr_id = '1'
online_candidate_res_root_dir=r'/zfssz2/ST_MCHRI/BIGDATA/PROJECT/NIPT_CNV/f_cnv_out/online/'+sample_id+'/cnv_call'
online_re_fname = 'Eval{}_{}_1000_200_out_cnv.csv'.format(sample_id, chr_id)
online_fn = os.path.join(online_candidate_res_root_dir, online_re_fname)

online_df = pd.read_csv(online_fn, sep='\t')
cols = ['POS_S', 'POS_E', 'LEN', 'P_NEU', 'P_DEL', 'P_DUP', 'PRED_L',
       'ID', 'POS', 'END', 'SVLEN', 'ALT', 'SMP_GT', 'ADJTYPE',
       'ADJAF', 'INTERVAL_LOWER', 'INTERVAL_UPPER', 'INTERVAL_LEN',
       'UNION_LEN']
online_df=online_df[cols]
online_df.head()

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,SVLEN,ALT,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN
0,0,10000,10000,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1
1,10000,29000,19000,0.867668,0.015923,0.11641,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1
2,29000,29200,200,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1
3,29200,36400,7200,0.669690,0.038330,0.29198,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1
4,36400,36600,200,NaN,NaN,NaN,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1


In [66]:
# analyze ture cnv label
gtruth_cnv_base_dir = '/zfssz6/ST_MCHRI/BIGDATA/P18Z10200N0124/NIPT_CNV/NIPT_CNV/1k_cnvs_lab_feat'
gtruth_fn = os.path.join(gtruth_cnv_base_dir, 'ALL.wgs.mergedSV.v8.20130502.svs.genotypes.GRCh38.vcf.' +
        sample_id + '.cnvs.labels')
gtruth_cnv_pd = pd.read_csv(gtruth_fn, sep='\t')
gtruth_cnv_pd = gtruth_cnv_pd[gtruth_cnv_pd['CHROM']==chr_id]
print('the total number of cnvs for sample {} chr {} is {}'.format(sample_id, chr_id, gtruth_cnv_pd.shape[0]))

true_del_df = gtruth_cnv_pd[gtruth_cnv_pd['ADJTYPE']=='DEL']
true_del_len_arr = true_del_df['SVLEN'].values
print('>'* 4 +'where DEL is {}, length range {}-{}, median: {}'.format(
    true_del_df.shape[0], min(true_del_len_arr), max(true_del_len_arr), np.median(true_del_len_arr)))
true_dup_df = gtruth_cnv_pd[gtruth_cnv_pd['ADJTYPE']=='DUP']
true_dup_len_arr = true_dup_df['SVLEN'].values
print('>'* 4 +'where DUP is {}, length range {}-{}, median: {}'.format(
    true_dup_df.shape[0], min(true_dup_len_arr), max(true_dup_len_arr), np.median(true_dup_len_arr)))

# check truth cnv labels with unpredective region
true_unpredictive_df = online_df[(online_df['PRED_L']==-1) & (~online_df['ID'].isnull())]
uppred_true_lab_lst = np.unique(true_unpredictive_df['ID'].values)
# check all truth cnv labels
true_cnv_df = online_df[(online_df['PRED_L']!=-1) &(~online_df['ID'].isnull())]
pred_true_lab_lst = np.unique(true_cnv_df['ID'].values)

unpred_cnvs = set(uppred_true_lab_lst) - set(pred_true_lab_lst)
unpred_dict = dict()
unpred_dict['DEL']=[]
unpred_dict['DUP']=[]
for i in unpred_cnvs:
    i_lbl = gtruth_cnv_pd.loc[gtruth_cnv_pd['ID']==i, 'ADJTYPE'].values[0]
    unpred_dict[i_lbl].append(i)
print('CNVs that are located at non-read region: ',unpred_dict)

the total number of cnvs for sample NA12878 chr 1 is 107
>>>>where DEL is 93, length range 50-78879, median: 1280.0
>>>>where DUP is 14, length range 10023-121787, median: 28301.5
CNVs that are located at non-read region:  {'DEL': ['BI_GS_DEL1_B3_P0073_40', 'BI_GS_DEL1_B1_P0249_462'], 'DUP': []}


In [67]:
# find the positive (tp+fp) and false negative
pred_cnv_df = online_df[((~online_df['ID'].isnull())|
                         (online_df['PRED_L']==1)|
                         (online_df['PRED_L']==2)) &
                       (online_df['PRED_L']!=-1)]

pred_cnv_df['TRUE_L']=0
pred_cnv_df.loc[pred_cnv_df['ADJTYPE']=='DUP', 'TRUE_L']=2
pred_cnv_df.loc[pred_cnv_df['ADJTYPE']=='DEL', 'TRUE_L']=1

pred_cnv_df=pred_cnv_df[pred_cnv_df['LEN']>=1000]
# pred_cnv_df=pred_cnv_df[pred_cnv_df['SVLEN']>=1000]

print(pred_cnv_df.shape[0])
pred_cnv_df.head()


331


/zfssz2/ST_MCHRI/BIGDATA/USER/baiyong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/zfssz2/ST_MCHRI/BIGDATA/USER/baiyong/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,SVLEN,ALT,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,TRUE_L
7,37200,38200,1000,0.240318,0.001080,0.758602,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
23,72600,74800,2200,0.333351,0.626874,0.039776,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
262,783188,813188,30000,0.346972,0.020542,0.632486,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
265,827388,867388,40000,0.317285,0.027559,0.655155,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
379,1652588,1664588,12000,0.425765,0.103252,0.470983,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0


In [68]:
# pred_cnv_df[pred_cnv_df['ADJTYPE']=='DEL']

In [69]:
fn_df = pred_cnv_df[(pred_cnv_df['PRED_L']==0)&(~pred_cnv_df['ID'].isnull())]
tp_fp_df = pred_cnv_df[pred_cnv_df['PRED_L']!=0]
result = pd.merge(fn_df, tp_fp_df, how='inner', on='ID')
re_cols = ['PRED_L_x', 'ID', 'POS_x', 'END_x', 'SVLEN_x', 'TRUE_L_x', 'INTERVAL_LEN_x', 'POS_S_y', 'POS_E_y',
           'POS_S_x', 'POS_E_x', 'LEN_x',
       'LEN_y', 'PRED_L_y', 'POS_y', 'END_y',
       'SVLEN_y', 'TRUE_L_y', 'INTERVAL_LEN_y']
result=result[re_cols]
result.head()
# the result show that the same true cnv could be predicted to be NEU or CNVs
# so we have to remove those predicted as NEU ones because we only reported CNVs rather than NEUs

,PRED_L_x,ID,POS_x,END_x,SVLEN_x,TRUE_L_x,INTERVAL_LEN_x,POS_S_y,POS_E_y,POS_S_x,POS_E_x,LEN_x,LEN_y,PRED_L_y,POS_y,END_y,SVLEN_y,TRUE_L_y,INTERVAL_LEN_y
0,0,DUP_gs_CNV_1_17026064_17052112,16699569.0,16725617.0,26048.0,2,233,16699384,16725384,16725384,16763384,38000,26000,2,16699569.0,16725617.0,26048.0,2,25815
1,0,DUP_gs_CNV_1_25592642_25661222,25266151.0,25334731.0,68580.0,1,26012,25294163,25306163,25258163,25292163,34000,12000,1,25266151.0,25334731.0,68580.0,1,12000
2,0,DUP_gs_CNV_1_25592642_25661222,25266151.0,25334731.0,68580.0,1,2000,25294163,25306163,25292163,25294163,2000,12000,1,25266151.0,25334731.0,68580.0,1,12000
3,0,DUP_gs_CNV_1_25592642_25661222,25266151.0,25334731.0,68580.0,1,28568,25294163,25306163,25306163,25338163,32000,12000,1,25266151.0,25334731.0,68580.0,1,12000
4,0,DUP_gs_CNV_1_85980461_86006144,85514778.0,85540461.0,25683.0,2,1226,85513235,85539235,85539235,85748635,209400,26000,2,85514778.0,85540461.0,25683.0,2,24457


In [70]:
adj_fn_df = fn_df[~fn_df.ID.isin(tp_fp_df.ID)]

In [71]:
result2 = pd.merge(adj_fn_df, tp_fp_df, how='inner', on='ID')
re_cols = ['PRED_L_x', 'ID', 'POS_x', 'END_x', 'SVLEN_x', 'TRUE_L_x', 'INTERVAL_LEN_x', 'POS_S_y', 'POS_E_y',
           'POS_S_x', 'POS_E_x', 'LEN_x',
       'LEN_y', 'PRED_L_y', 'POS_y', 'END_y',
       'SVLEN_y', 'TRUE_L_y', 'INTERVAL_LEN_y']
result2=result2[re_cols]
result2.head()

,PRED_L_x,ID,POS_x,END_x,SVLEN_x,TRUE_L_x,INTERVAL_LEN_x,POS_S_y,POS_E_y,POS_S_x,POS_E_x,LEN_x,LEN_y,PRED_L_y,POS_y,END_y,SVLEN_y,TRUE_L_y,INTERVAL_LEN_y


In [72]:
adj_fn_df[adj_fn_df['ID']=='DUP_gs_CNV_1_25592642_25661222']

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,SVLEN,ALT,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,TRUE_L


In [73]:
tp_fp_df[tp_fp_df['ID']=='DUP_gs_CNV_1_25592642_25661222']

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,SVLEN,ALT,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,TRUE_L
1152,25294163,25306163,12000,0.276193,0.497091,0.226716,1,DUP_gs_CNV_1_25592642_25661222,25266151.0,25334731.0,68580.0,"<CN0>,<CN2>","NA12878,1|0",DEL,0.20607,25294163,25306163,12000,68580,1


In [74]:
tn_df = pred_cnv_df[(pred_cnv_df['PRED_L']==0)&(pred_cnv_df['ID'].isnull())]

In [75]:
out_pred = pd.concat([adj_fn_df, tp_fp_df])



out_pred.loc[(np.abs(out_pred['P_DUP']-out_pred['P_NEU'])<0.1)&(out_pred['PRED_L']==2),'PRED_L']=0

In [76]:
tmp_df = out_pred[['P_NEU','P_DEL','P_DUP']]
tmp_df.columns = ['0','1','2']
out_pred['PRED_P_L']=tmp_df.idxmax(axis = 1, skipna = True)
out_pred['PRED_P_L']= out_pred['PRED_P_L'].astype(int)

In [77]:
out_pred.loc[(out_pred['PRED_L']!=0)&(out_pred['PRED_P_L']==0),'PRED_L']=0

In [78]:
out_pred=out_pred[out_pred['LEN']>1000]

In [79]:
min(out_pred['LEN'].values), max(out_pred['LEN'].values), np.median(out_pred['LEN'].values)

(1200, 1804000, 14000.0)

In [80]:
false_dup_df = out_pred[(out_pred['PRED_L']==2) & (out_pred['ID'].isnull())]
false_del_df = out_pred[(out_pred['PRED_L']==1) & (out_pred['ID'].isnull())]
false_neg_del_df = out_pred[(out_pred['PRED_L']==0) & (out_pred['TRUE_L']==1)]
false_neg_dup_df = out_pred[(out_pred['PRED_L']==0) & (out_pred['TRUE_L']==2)]
tp_del_df = out_pred[(out_pred['PRED_L']==1) & (out_pred['TRUE_L']==1)]
tp_dup_df = out_pred[(out_pred['PRED_L']==2) & (out_pred['TRUE_L']==2)]

In [81]:
len(false_dup_df), len(false_del_df), len(false_neg_del_df), len(false_neg_dup_df), len(tp_del_df), len(tp_dup_df)

(70, 61, 95, 3, 4, 12)

In [83]:
tp_dup_df

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,...,ALT,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,TRUE_L,PRED_P_L
980,16699384,16725384,26000,0.175081,0.005027,0.819892,2,DUP_gs_CNV_1_17026064_17052112,16699569.0,16725617.0,...,"<CN2>,<CN3>,<CN4>,<CN5>","NA12878,2|0",DUP,0.439696,16699569,16725384,25815,26233,2,2
2836,85513235,85539235,26000,0.209098,0.004008,0.786894,2,DUP_gs_CNV_1_85980461_86006144,85514778.0,85540461.0,...,"<CN2>,<CN3>","NA12878,1|1",DUP,0.987819,85514778,85539235,24457,27226,2,2
3939,120252635,120304635,52000,0.398992,0.050932,0.550076,2,DUP_gs_CNV_1_144691202_144710622,120286666.0,120306110.0,...,"<CN2>,<CN3>","NA12878,1|1",DUP,0.685703,120286666,120304635,17969,53475,2,2
17724,143735187,143789187,54000,0.294368,0.034157,0.671475,2,DUP_gs_CNV_1_149231192_149241216,143736542.0,143746565.0,...,"<CN2>,<CN3>","NA12878,1|1",DUP,0.875200,143736542,143746565,10023,54000,2,2
17725,143735187,143789187,54000,0.294368,0.034157,0.671475,2,DUP_gs_CNV_1_149249429_149284877,143754780.0,143790242.0,...,"<CN0>,<CN2>,<CN3>","NA12878,2|2",DUP,0.888379,143754780,143789187,34407,55055,2,2
17728,143797187,143865187,68000,0.394940,0.019083,0.585977,2,DUP_gs_CNV_1_149289438_149358445,143794803.0,143863898.0,...,"<CN2>,<CN3>","NA12878,1|1",DUP,0.883586,143797187,143863898,66711,70384,2,2
17904,144909587,144965587,56000,0.368540,0.019215,0.612246,2,DUP_gs_CNV_1_143984622_144094733,144915702.0,145023143.0,...,"<CN2>,<CN3>","NA12878,1|1",DUP,0.956669,144915702,144965587,49885,113556,2,2
17915,144968587,145028587,60000,0.341656,0.024332,0.634012,2,DUP_gs_CNV_1_143984622_144094733,144915702.0,145023143.0,...,"<CN2>,<CN3>","NA12878,1|1",DUP,0.956669,144968587,145023143,54556,112885,2,2
19802,148149987,148233987,84000,0.159895,0.007355,0.832750,2,DUP_uwash_chr1_146081807_146164363,148151364.0,148233918.0,...,"<CN2>,<CN3>","NA12878,1|1",DUP,0.999401,148151364,148233918,82554,84000,2,2
19813,148836187,149022187,186000,0.414448,0.026156,0.559397,2,DUP_gs_CNV_1_144973077_144989257,148895234.0,148911421.0,...,<CN2>,"NA12878,1|1",DUP,0.996805,148895234,148911421,16187,186000,2,2


In [84]:
tp_del_df

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,...,ALT,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,TRUE_L,PRED_P_L
1152,25294163,25306163,12000,0.276193,0.497091,0.226716,1,DUP_gs_CNV_1_25592642_25661222,25266151.0,25334731.0,...,"<CN0>,<CN2>","NA12878,1|0",DEL,0.206070,25294163,25306163,12000,68580,1,1
3736,112290035,112292635,2600,0.348939,0.576467,0.074594,1,YL_CN_ACB_172,112292274.0,112295202.0,...,<CN0>,"NA12878,1|1",DEL,0.873403,112292274,112292635,361,5167,1,1
3949,120953635,120965635,12000,0.349360,0.467999,0.182641,1,BI_GS_CNV_1_144468090_144480623,120953580.0,120966121.0,...,<CN0>,"NA12878,0|1",DEL,0.385383,120953635,120965635,12000,12541,1,1
22841,248631564,248633564,2000,0.153798,0.711095,0.135107,1,DUP_gs_CNV_1_248756741_248797597,248593440.0,248634296.0,...,"<CN0>,<CN2>","NA12878,1|1",DEL,0.173323,248631564,248633564,2000,40856,1,1


In [82]:
false_dup_df

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,...,ALT,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,TRUE_L,PRED_P_L
262,783188,813188,30000,0.346972,0.020542,0.632486,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,2
265,827388,867388,40000,0.317285,0.027559,0.655155,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,2
466,2578788,2580788,2000,0.266467,0.023163,0.710370,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,2
923,13658584,13660584,2000,0.322792,0.130948,0.546259,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,2
958,16019984,16063984,44000,0.402454,0.022810,0.574736,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,2
963,16473984,16477984,4000,0.311687,0.068373,0.619940,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,2
965,16489984,16498584,8600,0.337238,0.029077,0.633685,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,2
985,16881163,16899163,18000,0.226137,0.032955,0.740908,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,2
1004,19282163,19288163,6000,0.177952,0.055110,0.766938,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,2
1279,30351035,30359035,8000,0.297260,0.033189,0.669551,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,2


In [15]:
len(false_dup_df), len(false_del_df), len(false_neg_del_df), len(false_neg_dup_df), len(tp_del_df), len(tp_dup_df)

(107, 87, 96, 2, 5, 13)

In [16]:
print(min(false_dup_df['LEN'].values), max(false_dup_df['LEN'].values), np.median(false_dup_df['LEN'].values))
print(np.percentile(false_dup_df['LEN'].values, [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]))

1000 164000 8600.0
[  1000.   2000.   2000.   4000.   6000.   8600.  14000.  23200.  30160.
  53600. 164000.]


In [17]:
print(min(false_del_df['LEN'].values), max(false_del_df['LEN'].values), np.median(false_del_df['LEN'].values))
print(np.percentile(false_del_df['LEN'].values, [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]))

1000 8000 2000.0
[1000. 1000. 1200. 1400. 1800. 2000. 2000. 2000. 2360. 3600. 8000.]


In [18]:
print(min(false_neg_del_df['LEN'].values), max(false_neg_del_df['LEN'].values), np.median(false_neg_del_df['LEN'].values))
print(np.percentile(false_neg_del_df['LEN'].values, [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]))

1000 1804000 190700.0
[1.000e+03 1.820e+04 5.000e+04 6.800e+04 1.070e+05 1.907e+05 3.798e+05
 4.911e+05 5.900e+05 8.208e+05 1.804e+06]


In [19]:
print(min(false_neg_dup_df['LEN'].values), max(false_neg_dup_df['LEN'].values), np.median(false_neg_dup_df['LEN'].values))
print(np.percentile(false_neg_dup_df['LEN'].values, [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]))

88000 98400 93200.0
[88000. 89040. 90080. 91120. 92160. 93200. 94240. 95280. 96320. 97360.
 98400.]


In [20]:
false_dup_df

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,SVLEN,ALT,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,TRUE_L
7,37200,38200,1000,0.240318,0.001080,0.758602,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
262,783188,813188,30000,0.346972,0.020542,0.632486,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
265,827388,867388,40000,0.317285,0.027559,0.655155,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
379,1652588,1664588,12000,0.425765,0.103252,0.470983,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
466,2578788,2580788,2000,0.266467,0.023163,0.710370,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
646,6455090,6457090,2000,0.494986,0.141832,0.363182,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
903,13293984,13298184,4200,0.322759,0.257186,0.420055,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
923,13658584,13660584,2000,0.322792,0.130948,0.546259,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
958,16019984,16063984,44000,0.402454,0.022810,0.574736,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
963,16473984,16477984,4000,0.311687,0.068373,0.619940,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0


In [250]:
# f_out_pred = out_pred[out_pred['LEN']>=10000]
f_out_pred = out_pred[(out_pred['PRED_L']!=0)&(out_pred['LEN']>=4000)]
f_out_pred.head()

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,SVLEN,ALT,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,TRUE_L
262,783188,813188,30000,0.346972,0.020542,0.632486,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
265,827388,867388,40000,0.317285,0.027559,0.655155,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
379,1652588,1664588,12000,0.425765,0.103252,0.470983,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
790,12866890,12874890,8000,0.447621,0.500936,0.051442,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
903,13293984,13298184,4200,0.322759,0.257186,0.420055,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0


In [248]:
pred_lbl = f_out_pred['PRED_L'].values
true_lbl = f_out_pred['TRUE_L'].values

In [249]:
from pycm import ConfusionMatrix
confusion_matrix1 = ConfusionMatrix(true_lbl, pred_lbl)
#confusion_matrix.print_stats()
print(confusion_matrix1)

Predict  0        1        2        
Actual
0        0        8        80       

1        0        2        3        

2        0        0        13       





Overall Statistics : 

95% CI                                                            (0.07516,0.20786)
ACC Macro                                                         0.42767
AUNP                                                              0.51416
AUNU                                                              0.57139
Bennett S                                                         -0.28774
CBA                                                               0.11181
Chi-Squared                                                       None
Chi-Squared DF                                                    4
Conditional Entropy                                               0.41066
Cramer V                                                          None
Cross Entropy                                                     0.17819
F1

In [184]:
region_size = [(0, 10000), (10000, 100000), (100000, 10000000)]

for r_s, r_e in region_size:
    i_adj_fn_df = adj_fn_df[(adj_fn_df.LEN<=r_e)&(adj_fn_df.LEN>r_s)]
    
    i_tp_fp_df = tp_fp_df[(tp_fp_df.LEN<=r_e)&(tp_fp_df.LEN>r_s)]
    i_tp_df = i_tp_fp_df[i_tp_fp_df['PRED_L']==i_tp_fp_df['TRUE_L']]
    print('='*10)
    print(i_adj_fn_df)
    print('region:{}'.format(r_e))
    print('fn:{}, tp: {}, tp+fp:{}'.format(len(i_adj_fn_df), len(i_tp_df), len(i_tp_fp_df)))
    print('recall:{}, precision:{}'.format(len(i_tp_df)/(len(i_adj_fn_df)+len(i_tp_df)), len(i_tp_df)/len(i_tp_fp_df)))

           POS_S      POS_E   LEN     P_NEU     P_DEL     P_DUP  PRED_L  \
1962    59583635   59590635  7000  0.740127  0.191029  0.068844       0   
2141    67542635   67549235  6600  0.721704  0.229879  0.048417       0   
19956  152797387  152801387  4000  0.661578  0.048811  0.289611       0   
20857  189757387  189758387  1000  0.570330  0.429399  0.000271       0   
20862  189759187  189760387  1200  0.653909  0.343654  0.002437       0   
22232  236387164  236389164  2000  0.585081  0.196510  0.218408       0   

                            ID          POS          END    SVLEN    ALT  \
1962               UW_VH_10127   59582956.0   59583983.0   1027.0  <CN0>   
2141     BI_GS_DEL1_B3_P0069_2   67542287.0   67543128.0    841.0  <CN0>   
19956              UW_VH_24538  152787608.0  152798071.0  10463.0  <CN0>   
20857  BI_GS_DEL1_B4_P0190_157  189735347.0  189814226.0  78879.0  <CN0>   
20862  BI_GS_DEL1_B4_P0190_157  189735347.0  189814226.0  78879.0  <CN0>   
22232            Y

In [94]:
tp_fp_df[tp_fp_df['LEN']>=1000]

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,SVLEN,ALT,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN,TRUE_L
7,37200,38200,1000,0.240318,0.001080,0.758602,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
23,72600,74800,2200,0.333351,0.626874,0.039776,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
262,783188,813188,30000,0.346972,0.020542,0.632486,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
265,827388,867388,40000,0.317285,0.027559,0.655155,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
379,1652588,1664588,12000,0.425765,0.103252,0.470983,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
466,2578788,2580788,2000,0.266467,0.023163,0.710370,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
646,6455090,6457090,2000,0.494986,0.141832,0.363182,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
689,7413890,7414890,1000,0.224877,0.774954,0.000169,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
698,7601690,7603690,2000,0.325940,0.558145,0.115915,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0
790,12866890,12874890,8000,0.447621,0.500936,0.051442,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0


In [26]:
18/331

0.054380664652567974

In [9]:
x=pred_cnv_df[pred_cnv_df['LEN']>50000]
x

,POS_S,POS_E,LEN,P_NEU,P_DEL,P_DUP,PRED_L,ID,POS,END,SVLEN,SVTYPE,AF,SMP_GT,ADJTYPE,ADJAF,INTERVAL_LOWER,INTERVAL_UPPER,INTERVAL_LEN,UNION_LEN
605,4067290,4654290,587000,0.849145,0.043462,0.107393,0,DEL_pindel_129,4144607.0,4144657.0,50.0,DEL,0.485623,"NA12878,1|0",DEL,0.485623,4144607,4144657,50,587000
692,7417090,7598090,181000,0.888384,0.024985,0.086631,0,BI_GS_DEL1_B5_P0008_654,7510014.0,7511461.0,1447.0,DEL,0.161741,"NA12878,1|0",DEL,0.161741,7510014,7511461,1447,181000
737,9470090,9588090,118000,0.855709,0.065640,0.078651,0,YL_CN_GBR_48,9535301.0,9537810.0,2509.0,DEL,0.219249,"NA12878,1|0",DEL,0.219249,9535301,9537810,2509,118000
750,10258490,10638290,379800,0.843745,0.064849,0.091405,0,UW_VH_0,10422442.0,10423733.0,1291.0,DEL,0.329273,"NA12878,0|1",DEL,0.329273,10422442,10423733,1291,379800
780,11168090,11654090,486000,0.877115,0.046888,0.075997,0,UW_VH_58,11622816.0,11623132.0,316.0,DEL,0.505591,"NA12878,1|0",DEL,0.505591,11622816,11623132,316,486000
952,15758584,15847384,88800,0.850868,0.090585,0.058547,0,YL_CN_FIN_64,15825187.0,15829137.0,3950.0,DEL,0.23742,"NA12878,1|0",DEL,0.237420,15825187,15829137,3950,88800
1048,21410363,21466363,56000,0.569909,0.063159,0.366931,0,BI_GS_DEL1_B3_P0022_20,21459925.0,21460202.0,277.0,DEL,0.713858,"NA12878,1|1",DEL,0.713858,21459925,21460202,277,56000
1156,25370163,25426163,56000,0.417517,0.066450,0.516033,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,-1,-1,-1
1307,30684035,30908035,224000,0.885863,0.035542,0.078594,0,DEL_pindel_417,30771803.0,30771897.0,94.0,DEL,0.315895,"NA12878,0|1",DEL,0.315895,30771803,30771897,94,224000
1309,30908235,31102435,194200,0.855976,0.065362,0.078661,0,SI_BD_281,31102087.0,31104352.0,2265.0,DEL,0.00139776,"NA12878,1|0",DEL,0.001398,31102087,31102435,348,196117
